In [210]:
import pandas as pd
import plotly
import json
import plotly.express as px
import geopandas
import folium

In [247]:
animals = pd.read_csv("animals.csv")
animals["entity_id"] = animals["entity_id"].str.split(":").str[3]
animals["location"] = geopandas.GeoSeries.from_wkt(animals["location"])
tractors = pd.read_csv("tractors.csv")
soil = pd.read_csv("soilsensor.csv")
filling = pd.read_csv("fillingsensor.csv")

# Graficas propias

Porcentaje de status de cada animal de la granja con un barplot

In [74]:
# Status por animal de las granjas barchart
count = animals[["entity_id", "status"]].groupby(["entity_id", "status"]).size().reset_index(name = "count")
fig_bar = px.bar(count, x="entity_id", y = "count", color="status")
fig_bar.show()

Boxplot de las pulsaciones de cerdos y vacas

In [53]:
fig = px.box(animals, x="entity_id", y="heartrate", points="all", color = "entity_id")
fig.show()

Pie-chart estados tractores

In [72]:
count2 = tractors[["entity_id", "status"]].groupby(["entity_id", "status"]).size().reset_index(name = "count")
total_count2 = count2.groupby('entity_id')['count'].transform('sum')
count2['percentage'] = (count['count'] / total_count) * 100
fig_pie = px.pie(count2, values="percentage", names="status", facet_col="entity_id")
fig_pie.show()

In [69]:
fig = px.histogram(soil, x="humidity", barmode="stack", facet_col="entity_id", color="entity_id")
fig.show()

# Graficas enunciado

Evolución filling

In [76]:
fig = px.line(filling, x='time_index', y="filling")
fig.show()

Evolución humedad y temperatura

In [77]:
fig = px.line(soil, x='time_index', y="humidity")
fig.show()

Evolución temporal del status de la primera vaca y del primer cerdo

In [80]:
primera_vaca = animals[animals['entity_id'] == "cow001"]
primer_cerdo = animals[animals['entity_id'] == "pig001"]

In [81]:
fig = px.line(primera_vaca, x='time_index', y="status")
fig.show()

In [82]:
fig = px.line(primer_cerdo, x='time_index', y="status")
fig.show()

Evolución heartrate primera vaca y primer cerdo

In [83]:
fig = px.line(primera_vaca, x='time_index', y="heartrate")
fig.show()

In [84]:
fig = px.line(primer_cerdo, x='time_index', y="heartrate")
fig.show()

Folium

In [252]:
import geopandas

animals_gdf = geopandas.GeoDataFrame(animals, geometry="location", crs = "epsg:4326")
last_gdf = geopandas.GeoDataFrame(animals.groupby('entity_id').last().reset_index(), geometry="location", crs = "epsg:4326")

In [227]:
last_gdf

,entity_id,time_index,heartrate,location,status
0,cow001,2024-05-08T16:56:21,52,POINT (13.41000 52.46900),6
1,cow002,2024-05-08T16:56:26,51,POINT (13.41200 52.46800),6
2,cow003,2024-05-08T16:56:26,51,POINT (13.41000 52.46900),0
3,cow004,2024-05-08T16:56:21,51,POINT (13.41100 52.46500),6
4,pig001,2024-05-08T16:56:21,61,POINT (13.35400 52.51700),0
5,pig002,2024-05-08T16:56:26,64,POINT (13.35400 52.51400),3
6,pig003,2024-05-08T16:56:26,62,POINT (13.35300 52.52000),0
7,pig004,2024-05-08T16:56:26,64,POINT (13.35300 52.51500),5


In [219]:
m = folium.Map()
folium.GeoJson(
    last_gdf, name="Animals",
    marker=folium.Marker(),
    tooltip=folium.GeoJsonTooltip(fields=["entity_id", "heartrate", "status"]),
    popup=folium.GeoJsonPopup(fields=["entity_id", "heartrate", "status"]),
    zoom_on_click=True,
).add_to(m)

In [220]:
m

In [253]:
# Lista de características GeoJSON
features = []
for idx, row in animals_gdf.iterrows():
    line_feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["location"].x, row["location"].y],
        },
        "properties": {
            "time": row["time_index"],
            "style": {
                "color": "blue",
                "weight": 5,
            },
        },
    }
    features.append(line_feature)

In [254]:
from folium.plugins import TimestampedGeoJson
m2 = folium.Map()
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="P1M",
    add_last_point=True,
).add_to(m2)

In [256]:
m2